## 1. Instalação de Dependências e Imports

In [2]:
# Instalação de bibliotecas necessárias
!pip install requests pandas numpy tabulate matplotlib --quiet

# Imports
import math
import requests
import json
import pandas as pd
import numpy as np
from tabulate import tabulate
from typing import Dict, Any, Optional

print("✅ Bibliotecas importadas com sucesso!")

✅ Bibliotecas importadas com sucesso!


## 2. Definição de Variáveis de Entrada

Dados extraídos do payload da requisição real ao endpoint MPPT.

In [3]:
# ==========================================
# DADOS DO MÓDULO FOTOVOLTAICO
# ==========================================
modulo = {
    "fabricante": "Fabricante Genérico",
    "modelo": "Módulo 550W",
    "potencia_nominal_w": 550,          # Potência nominal em Watts
    "voc_stc": 41.7,                     # Tensão de circuito aberto em STC (25°C) [Volts]
    "isc_stc": 13.8,                     # Corrente de curto-circuito em STC [Amperes]
    "temp_coef_voc": -0.27,              # Coeficiente de temperatura do Voc [%/°C]
    "temp_coef_isc": 0.05                # Coeficiente de temperatura do Isc [%/°C] (típico)
}

# ==========================================
# DADOS DO INVERSOR
# ==========================================
inversor = {
    "fabricante": "Fronius",
    "modelo": "Primo 15.0-1",
    "potencia_saida_ca_w": 15000,       # Potência nominal AC [Watts]
    "tensao_cc_max_v": 1000,             # Tensão DC máxima absoluta [Volts]
    "numero_mppt": 2,                    # Número de rastreadores MPPT
    "strings_por_mppt": 1,               # Strings por MPPT (configuração)
    "corrente_entrada_max_a": 20,        # Corrente máxima de entrada [Amperes]
    "faixa_mppt_min_v": 200,             # Tensão mínima MPPT [Volts]
    "faixa_mppt_max_v": 800,             # Tensão máxima MPPT [Volts]
    "tipo_rede": "Monofásico 220V"
}

# ==========================================
# LOCALIZAÇÃO (para consulta de temperatura)
# ==========================================
localizacao = {
    "latitude": -23.7617,                # Maringá, PR
    "longitude": -53.3294
}

# ==========================================
# PARÂMETROS DE CÁLCULO
# ==========================================
parametros = {
    "margem_seguranca_voc": 0.05,        # 5% de margem (NBR 16274)
    "fator_sobredimensionamento": 1.2,   # 120% da potência do inversor
    "temperatura_stc": 25.0,             # Temperatura STC em °C
    "temperatura_minima_default": 0.0    # Temperatura mínima padrão se PVGIS falhar
}

# Exibir resumo dos dados de entrada
print("="*60)
print("📋 DADOS DE ENTRADA")
print("="*60)
print(f"\n🔆 MÓDULO: {modulo['fabricante']} {modulo['modelo']}")
print(f"   Potência: {modulo['potencia_nominal_w']}W")
print(f"   Voc (STC): {modulo['voc_stc']}V")
print(f"   Isc (STC): {modulo['isc_stc']}A")
print(f"   Coef. Temp. Voc: {modulo['temp_coef_voc']}%/°C")

print(f"\n⚡ INVERSOR: {inversor['fabricante']} {inversor['modelo']}")
print(f"   Potência AC: {inversor['potencia_saida_ca_w']/1000:.1f}kW")
print(f"   Faixa MPPT: {inversor['faixa_mppt_min_v']}-{inversor['faixa_mppt_max_v']}V")
print(f"   Número de MPPTs: {inversor['numero_mppt']}")
print(f"   Strings por MPPT: {inversor['strings_por_mppt']}")
print(f"   Corrente máxima: {inversor['corrente_entrada_max_a']}A")

print(f"\n📍 LOCALIZAÇÃO:")
print(f"   Lat: {localizacao['latitude']}, Lon: {localizacao['longitude']}")
print("="*60)

📋 DADOS DE ENTRADA

🔆 MÓDULO: Fabricante Genérico Módulo 550W
   Potência: 550W
   Voc (STC): 41.7V
   Isc (STC): 13.8A
   Coef. Temp. Voc: -0.27%/°C

⚡ INVERSOR: Fronius Primo 15.0-1
   Potência AC: 15.0kW
   Faixa MPPT: 200-800V
   Número de MPPTs: 2
   Strings por MPPT: 1
   Corrente máxima: 20A

📍 LOCALIZAÇÃO:
   Lat: -23.7617, Lon: -53.3294


## 3. Funções Auxiliares

In [4]:
def consultar_temperatura_minima_pvgis(lat: float, lon: float) -> Optional[float]:
    """
    Consulta a API PVGIS para obter a temperatura mínima histórica.
    
    Args:
        lat: Latitude
        lon: Longitude
    
    Returns:
        Temperatura mínima em °C ou None se falhar
    """
    try:
        print("🌍 Consultando PVGIS para temperatura mínima...")
        
        # URL da API PVGIS TMY (Typical Meteorological Year)
        url = "https://re.jrc.ec.europa.eu/api/v5_2/tmy"
        
        params = {
            "lat": lat,
            "lon": lon,
            "outputformat": "json"
        }
        
        response = requests.get(url, params=params, timeout=30)
        
        if response.status_code == 200:
            data = response.json()
            
            # Extrair dados horários
            if "outputs" in data and "tmy_hourly" in data["outputs"]:
                tmy_data = data["outputs"]["tmy_hourly"]
                
                # Encontrar temperatura mínima
                temperaturas = [row["T2m"] for row in tmy_data]
                temp_min = min(temperaturas)
                
                print(f"   ✅ Temperatura mínima obtida: {temp_min:.1f}°C")
                print(f"   📊 Base de dados: PVGIS TMY (8760 horas)")
                
                return temp_min
        
        print("   ⚠️  PVGIS não disponível, usando valor padrão")
        return None
        
    except Exception as e:
        print(f"   ❌ Erro ao consultar PVGIS: {e}")
        return None


def calcular_voc_temperatura_minima(voc_stc: float, 
                                    temp_coef_voc: float, 
                                    temp_min: float, 
                                    temp_stc: float = 25.0) -> float:
    """
    Calcula Voc na temperatura mínima.
    
    Fórmula: Voc(T) = Voc(STC) × [1 + β_Voc × (T - T_STC)]
    
    Args:
        voc_stc: Voc em condições STC (25°C) [V]
        temp_coef_voc: Coeficiente de temperatura [%/°C]
        temp_min: Temperatura mínima [°C]
        temp_stc: Temperatura STC [°C]
    
    Returns:
        Voc na temperatura mínima [V]
    """
    # Converter coeficiente de % para decimal
    beta_voc = temp_coef_voc / 100.0
    
    # Diferença de temperatura
    delta_t = temp_min - temp_stc
    
    # Cálculo
    voc_frio = voc_stc * (1 + beta_voc * delta_t)
    
    return voc_frio


def aplicar_margem_seguranca(valor: float, margem: float) -> float:
    """
    Aplica margem de segurança ao valor.
    
    Args:
        valor: Valor base
        margem: Margem percentual (ex: 0.05 para 5%)
    
    Returns:
        Valor com margem aplicada
    """
    return valor * (1 + margem)


print("✅ Funções auxiliares definidas!")

✅ Funções auxiliares definidas!


## 4. Processamento - Cálculo MPPT

In [5]:
print("\n" + "="*60)
print("🔧 INICIANDO CÁLCULO MPPT")
print("="*60)

# ==========================================
# PASSO 1: Obter temperatura mínima
# ==========================================
print("\n📍 PASSO 1: Consulta de Temperatura Mínima")
print("-" * 60)

temp_min = consultar_temperatura_minima_pvgis(
    localizacao["latitude"], 
    localizacao["longitude"]
)

if temp_min is None:
    temp_min = parametros["temperatura_minima_default"]
    print(f"   🔄 Usando temperatura padrão: {temp_min}°C")

# ==========================================
# PASSO 2: Calcular Voc em temperatura fria
# ==========================================
print("\n🌡️  PASSO 2: Cálculo de Voc em Temperatura Fria")
print("-" * 60)

voc_frio = calcular_voc_temperatura_minima(
    modulo["voc_stc"],
    modulo["temp_coef_voc"],
    temp_min,
    parametros["temperatura_stc"]
)

# Cálculos intermediários para exibição
delta_t = temp_min - parametros["temperatura_stc"]
beta_voc = modulo["temp_coef_voc"] / 100.0
variacao_percentual = beta_voc * delta_t

print(f"   Fórmula: Voc(T) = Voc(STC) × [1 + β_Voc × (T_min - T_STC)]")
print(f"   ")
print(f"   Voc(STC) = {modulo['voc_stc']:.2f}V")
print(f"   β_Voc = {modulo['temp_coef_voc']}%/°C = {beta_voc:.4f}/°C")
print(f"   T_min = {temp_min:.1f}°C")
print(f"   T_STC = {parametros['temperatura_stc']:.1f}°C")
print(f"   ΔT = {delta_t:.1f}°C")
print(f"   ")
print(f"   Cálculo:")
print(f"   Voc_frio = {modulo['voc_stc']:.2f} × [1 + {beta_voc:.4f} × {delta_t:.1f}]")
print(f"   Voc_frio = {modulo['voc_stc']:.2f} × [1 + {variacao_percentual:.4f}]")
print(f"   Voc_frio = {modulo['voc_stc']:.2f} × {1 + variacao_percentual:.4f}")
print(f"   ")
print(f"   ✅ Voc a {temp_min:.1f}°C = {voc_frio:.2f}V (aumento de {variacao_percentual*100:.1f}%)")

# ==========================================
# PASSO 3: Aplicar margem de segurança
# ==========================================
print("\n🛡️  PASSO 3: Aplicação de Margem de Segurança (NBR 16274)")
print("-" * 60)

voc_seguro = aplicar_margem_seguranca(
    voc_frio,
    parametros["margem_seguranca_voc"]
)

margem_aplicada = voc_seguro - voc_frio

print(f"   Margem de segurança: {parametros['margem_seguranca_voc']*100:.0f}%")
print(f"   Razão: Proteção contra picos transitórios e envelhecimento")
print(f"   ")
print(f"   Voc_seguro = {voc_frio:.2f}V × 1.05")
print(f"   Voc_seguro = {voc_seguro:.2f}V")
print(f"   ")
print(f"   ✅ Margem adicionada: +{margem_aplicada:.2f}V")

# ==========================================
# PASSO 4: Calcular limite de tensão
# ==========================================
print("\n⚡ PASSO 4: Determinação do Limite por Tensão")
print("-" * 60)

max_modulos_serie_tensao = math.floor(
    inversor["faixa_mppt_max_v"] / voc_seguro
)

tensao_string_max = max_modulos_serie_tensao * voc_seguro
margem_tensao = inversor["faixa_mppt_max_v"] - tensao_string_max

print(f"   Tensão MPPT máxima: {inversor['faixa_mppt_max_v']}V")
print(f"   Voc seguro por módulo: {voc_seguro:.2f}V")
print(f"   ")
print(f"   Cálculo:")
print(f"   N_série_max = ⌊{inversor['faixa_mppt_max_v']} / {voc_seguro:.2f}⌋")
print(f"   N_série_max = ⌊{inversor['faixa_mppt_max_v'] / voc_seguro:.2f}⌋")
print(f"   N_série_max = {max_modulos_serie_tensao} módulos")
print(f"   ")
print(f"   Verificação:")
print(f"   V_string = {max_modulos_serie_tensao} × {voc_seguro:.2f}V = {tensao_string_max:.2f}V")
print(f"   Margem restante: {margem_tensao:.2f}V")
print(f"   ")
print(f"   ✅ Limite por tensão: {max_modulos_serie_tensao} módulos por string")

# ==========================================
# PASSO 5: Calcular limite de corrente
# ==========================================
print("\n🔌 PASSO 5: Determinação do Limite por Corrente")
print("-" * 60)

max_strings_paralelo_corrente = math.floor(
    inversor["corrente_entrada_max_a"] / modulo["isc_stc"]
)

# Respeitar configuração do inversor
max_strings_por_mppt = min(
    inversor["strings_por_mppt"],
    max_strings_paralelo_corrente
)

corrente_por_mppt = max_strings_por_mppt * modulo["isc_stc"]
margem_corrente = inversor["corrente_entrada_max_a"] - corrente_por_mppt

print(f"   Corrente MPPT máxima: {inversor['corrente_entrada_max_a']}A")
print(f"   Isc por módulo: {modulo['isc_stc']}A")
print(f"   Strings por MPPT (configuração): {inversor['strings_por_mppt']}")
print(f"   ")
print(f"   Cálculo:")
print(f"   N_strings_max = ⌊{inversor['corrente_entrada_max_a']} / {modulo['isc_stc']}⌋")
print(f"   N_strings_max = ⌊{inversor['corrente_entrada_max_a'] / modulo['isc_stc']:.2f}⌋")
print(f"   N_strings_max = {max_strings_paralelo_corrente} strings")
print(f"   ")
print(f"   Strings efetivas: min({inversor['strings_por_mppt']}, {max_strings_paralelo_corrente}) = {max_strings_por_mppt}")
print(f"   ")
print(f"   Verificação:")
print(f"   I_MPPT = {max_strings_por_mppt} × {modulo['isc_stc']}A = {corrente_por_mppt:.2f}A")

if corrente_por_mppt > inversor["corrente_entrada_max_a"]:
    print(f"   ❌ INCOMPATIBILIDADE: Corrente ({corrente_por_mppt:.2f}A) > Máxima ({inversor['corrente_entrada_max_a']}A)")
    compatibilidade_corrente = False
else:
    print(f"   Margem restante: {margem_corrente:.2f}A")
    print(f"   ✅ Limite por corrente: {max_strings_por_mppt} string(s) por MPPT")
    compatibilidade_corrente = True

# ==========================================
# PASSO 6: Calcular limite de potência
# ==========================================
print("\n💡 PASSO 6: Determinação do Limite por Potência")
print("-" * 60)

potencia_desejada = inversor["potencia_saida_ca_w"] * parametros["fator_sobredimensionamento"]
total_modulos_potencia = math.ceil(potencia_desejada / modulo["potencia_nominal_w"])

print(f"   Potência nominal inversor: {inversor['potencia_saida_ca_w']/1000:.1f}kW")
print(f"   Fator de sobredimensionamento: {parametros['fator_sobredimensionamento']*100:.0f}%")
print(f"   Potência módulo: {modulo['potencia_nominal_w']}W")
print(f"   ")
print(f"   Cálculo:")
print(f"   P_desejada = {inversor['potencia_saida_ca_w']} × {parametros['fator_sobredimensionamento']}")
print(f"   P_desejada = {potencia_desejada:.0f}W")
print(f"   ")
print(f"   N_total = ⌈{potencia_desejada:.0f} / {modulo['potencia_nominal_w']}⌉")
print(f"   N_total = ⌈{potencia_desejada / modulo['potencia_nominal_w']:.2f}⌉")
print(f"   N_total = {total_modulos_potencia} módulos")
print(f"   ")
print(f"   ✅ Limite por potência: {total_modulos_potencia} módulos totais")

# ==========================================
# PASSO 7: Configuração final do sistema
# ==========================================
print("\n⚙️  PASSO 7: Configuração Final do Sistema")
print("-" * 60)

if not compatibilidade_corrente:
    print("   ❌ SISTEMA INCOMPATÍVEL: Corrente do módulo excede limite do inversor")
    print("   ")
    print("   Soluções possíveis:")
    print(f"   1. Usar módulo com Isc ≤ {inversor['corrente_entrada_max_a']}A")
    print(f"   2. Usar inversor com corrente MPPT ≥ {modulo['isc_stc']}A")
else:
    # Calcular distribuição
    num_strings_total = inversor["numero_mppt"] * max_strings_por_mppt
    modulos_por_string = math.ceil(total_modulos_potencia / num_strings_total)
    
    # Ajustar ao limite de tensão
    if modulos_por_string > max_modulos_serie_tensao:
        modulos_por_string = max_modulos_serie_tensao
        print(f"   ⚠️  Ajustado ao limite de tensão")
    
    # Totais finais
    total_modulos_final = num_strings_total * modulos_por_string
    potencia_instalada = total_modulos_final * modulo["potencia_nominal_w"]
    taxa_sobredimensionamento = potencia_instalada / inversor["potencia_saida_ca_w"]
    
    print(f"   Número de MPPTs: {inversor['numero_mppt']}")
    print(f"   Strings por MPPT: {max_strings_por_mppt}")
    print(f"   Total de strings: {num_strings_total}")
    print(f"   ")
    print(f"   Cálculo de distribuição:")
    print(f"   Módulos por string = ⌈{total_modulos_potencia} / {num_strings_total}⌉ = {modulos_por_string}")
    print(f"   ")
    print(f"   ✅ CONFIGURAÇÃO FINAL:")
    print(f"   • Módulos por string: {modulos_por_string}")
    print(f"   • Total de módulos: {total_modulos_final}")
    print(f"   • Potência instalada: {potencia_instalada/1000:.2f}kWp")
    print(f"   • Sobredimensionamento: {taxa_sobredimensionamento*100:.1f}%")
    
    # Verificações finais
    tensao_string_final = modulos_por_string * voc_seguro
    corrente_mppt_final = max_strings_por_mppt * modulo["isc_stc"]
    
    print(f"   ")
    print(f"   Verificações de segurança:")
    print(f"   • Tensão string: {tensao_string_final:.2f}V < {inversor['faixa_mppt_max_v']}V ✅")
    print(f"   • Corrente MPPT: {corrente_mppt_final:.2f}A < {inversor['corrente_entrada_max_a']}A ✅")
    print(f"   • Potência instalada: {potencia_instalada/1000:.2f}kWp ≈ {potencia_desejada/1000:.2f}kWp ✅")

print("\n" + "="*60)
print("✅ CÁLCULO CONCLUÍDO")
print("="*60)


🔧 INICIANDO CÁLCULO MPPT

📍 PASSO 1: Consulta de Temperatura Mínima
------------------------------------------------------------
🌍 Consultando PVGIS para temperatura mínima...
   ✅ Temperatura mínima obtida: 7.3°C
   📊 Base de dados: PVGIS TMY (8760 horas)

🌡️  PASSO 2: Cálculo de Voc em Temperatura Fria
------------------------------------------------------------
   Fórmula: Voc(T) = Voc(STC) × [1 + β_Voc × (T_min - T_STC)]
   
   Voc(STC) = 41.70V
   β_Voc = -0.27%/°C = -0.0027/°C
   T_min = 7.3°C
   T_STC = 25.0°C
   ΔT = -17.7°C
   
   Cálculo:
   Voc_frio = 41.70 × [1 + -0.0027 × -17.7]
   Voc_frio = 41.70 × [1 + 0.0478]
   Voc_frio = 41.70 × 1.0478
   
   ✅ Voc a 7.3°C = 43.69V (aumento de 4.8%)

🛡️  PASSO 3: Aplicação de Margem de Segurança (NBR 16274)
------------------------------------------------------------
   Margem de segurança: 5%
   Razão: Proteção contra picos transitórios e envelhecimento
   
   Voc_seguro = 43.69V × 1.05
   Voc_seguro = 45.88V
   
   ✅ Margem adicio